In [13]:
from collections import defaultdict
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
import xmltodict

root_directory = os.path.join(os.getcwd()) # '/Users/Username/dataset'
images_directory = os.path.join(root_directory, "images")
masks_directory = os.path.join(root_directory, "annotations", "trimaps")
xml_directory = os.path.join(root_directory, "annotations", "xmls")

# Dir_name 
# [GREY]Grey Scale - rgb2gray
# [SILO]실루엣 - 고양이 흰색 그외 검정색
# [TXTR]텍스쳐 - 바운딩 박스 크롭하고 그 중 Half
# [BACK]배경 - 고양이 검정색 배경 그대로

aug_dir_names = ['grey', 'silo', 'txtr', 'back']
for name in aug_dir_names:
    dir_name = os.path.join(root_directory, f'images_{name}')
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

images_filenames = list(sorted(os.listdir(images_directory)))
correct_images_filenames = [i for i in images_filenames if cv2.imread(os.path.join(images_directory, i)) is not None]
correct_images_filenames = [i for i in correct_images_filenames if os.path.splitext(i)[1] == '.jpg']

In [14]:
xml_directory

'/Users/woolee/biases-are-features-dataset/annotations/xmls'

In [4]:

xml_file = '/Users/woolee/biases-are-features-dataset/annotations/xmls/Abyssinian_1.xml'
f = open(xml_file)
doc = xmltodict.parse(f.read()) 

In [12]:
xmin = doc['annotation']['object']['bndbox']['xmin']
ymin = doc['annotation']['object']['bndbox']['ymin']
xmax = doc['annotation']['object']['bndbox']['xmax']
ymax = doc['annotation']['object']['bndbox']['ymax']
print(xmin, ymin, xmax, ymax)

333 72 425 158


In [11]:
doc['annotation']['object']['bndbox']

'''
OrderedDict([('xmin', '333'),
             ('ymin', '72'),
             ('xmax', '425'),
             ('ymax', '158')])
'''

OrderedDict([('xmin', '333'),
             ('ymin', '72'),
             ('xmax', '425'),
             ('ymax', '158')])

In [30]:
"""## TXTR"""
def get_bb(image_filename):
    '''
    About: Finding Bounding Box for each image, using xml file. If no xml file found, just return zeros.
    Input: Image_filename
    Output: xmin, ymin, xmax, ymax (location of bb)
    '''
    xml_dir = os.path.join(xml_directory, image_filename.replace(".jpg", ".xml"))
    try:
        f = open(xml_dir)
        doc = xmltodict.parse(f.read()) 
        xmin = int(doc['annotation']['object']['bndbox']['xmin'])
        ymin = int(doc['annotation']['object']['bndbox']['ymin'])
        xmax = int(doc['annotation']['object']['bndbox']['xmax'])
        ymax = int(doc['annotation']['object']['bndbox']['ymax'])

    except:
        xmin, ymin, xmax, ymax = 0,0,0,0

    return xmin, ymin, xmax, ymax

def to_txtr(images_filenames, images_directory, target_directory):
    for i, image_filename in enumerate(images_filenames[0:10]):
        extension = os.path.splitext(image_filename)[1] # find extension to exclue '.mat'
        if (extension == '.jpg'):
            image = cv2.imread(os.path.join(images_directory, image_filename))
            w, h = image.shape[0], image.shape[1]
            xmin, ymin, xmax, ymax = get_bb(image_filename)
            
        if xmin == 0 and ymin == 0 and xmax == 0 and ymax == 0:
            xmax = w
            ymax = h

        txtred_image = image[ymin + int((ymax-ymin)/4): ymax - int((ymax-ymin)/4),\
                         xmin + int((xmax-xmin)/4): xmax - int((xmax-xmin)/4)]
        txtred_image = cv2.resize(txtred_image, dsize = (h, w), interpolation = cv2.INTER_LINEAR)
        cv2.imwrite(os.path.join(target_directory , image_filename), txtred_image)

to_txtr(correct_images_filenames, images_directory, os.path.join(root_directory, "images_txtr"))

In [2]:
"""## TXTR"""

def preprocess_mask_txtr(mask):
    mask = mask.astype(np.uint8)
    mask[(mask == 2.0) | (mask == 3.0)] = 0.0
    mask[mask == 1.0] = 255
    return mask

def to_txtr(images_filenames, images_directory, target_directory):
    for i, image_filename in enumerate(images_filenames):
        extension = os.path.splitext(image_filename)[1] # find extension to exclue '.mat'
        if (extension == '.jpg'):
            image = cv2.imread(os.path.join(images_directory, image_filename))
            mask = cv2.imread(os.path.join(masks_directory, image_filename.replace(".jpg", ".png")), cv2.IMREAD_UNCHANGED,)
        _,thresh = cv2.threshold(mask,1,255,0)
        contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # contours: (1, 4, 1, 2)
        bbs = []
        for contour in contours:
            bb = cv2.boundingRect(contour)
            bbs.append(bb)

        # bbs 맨뒤의 요소: 이미지 전체 박스 지칭 (0,0,w,h)
        # bbs 맨뒤에서 두번째 요소: Mask 지칭
        
        if len(bbs) > 1:
            x,y,w,h = bbs[-2]
            if not w > 50 or h > 50: # 이미지 bb가 지극히 작아서 (= 너비와 높이가 50픽셀도 안되어서) BB를 Crop하면 texture는 커녕 점의 형태에 가까운 이미지를 Return합니다. 이럴경우 그냥 이미지 전체에 가운데 1/2만 뽑아내도록 구현했습니다.
                x,y,w,h = bbs[-1]
                x += int(w/4)
                y += int(h/4)
                w = int(w/2)
                h = int(h/2)

        else:
            x,y,w,h = bbs[-1]

        txtred_image = image[y + int(h/4): y + int(3*h/4), x + int(w/4): x + int(3*w/4)]
        cv2.imwrite(os.path.join(target_directory , image_filename), txtred_image)

# to_txtr(correct_images_filenames, images_directory, os.path.join(root_directory, "images_txtr"))